In [116]:
from dash import Dash, html, dcc, callback, Output, Input, State
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
from dash import dash_table
import plotly_express as px
import pandas as pd

In [117]:
os_data = pd.read_csv("data/athlete_events.csv")
os_data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [245]:
app = Dash(
    # får namnet på filen
    __name__, 
    # QUARTZ ändras till det temat man vill ha på bakgrundsfärg tex. Se på https://bootswatch.com/quartz/
    external_stylesheets=[dbc.themes.DARKLY],
    # Säger till webbläsaren att siten är responsive (ändrar sig på mindre enheter)
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            # Skapar en rubrik med angivet textinnehåll, classname? mt-3 gör att det blir marginal ovanför rubriken på siten, "text-center" centrerar innehållet i kolumnen. "text-primary" i parantesen om väla förvald färg, "primary" byts ut mot den färgen man vill använda. Länk för förvalda färger: https://bootswatch.com/darkly/
            html.H1("Olympics", className="text-center mt-3", style={"color":"#FFFFFF", "font-family": "Livre Baskerville"})
            # width= Kolumnen sträcker sig över hela skärmen.Här ändras with för att bestämma hur presenteras på mobilen för large, medium och smal mobiler osv.
        ], width=12)
    ]), 

    dbc.Row([
        dbc.Col([
            # Skapar en rubrik med angivet textinnehåll, classname? mt-3 gör att det blir marginal ovanför rubriken på siten, "text-center" centrerar innehållet i kolumnen. "text-primary" i parantesen om väla förvald färg, "primary" byts ut mot den färgen man vill använda. Länk för förvalda färger: https://bootswatch.com/darkly/
            html.H2("Dataanalysis", className="text-center mt-3 mb-3", style={"color":"#FFFFFF", "font-family": "Livre Baskerville"})
            # width= Kolumnen sträcker sig över hela skärmen.Här ändras with för att bestämma hur presenteras på mobilen för large, medium och smal mobiler osv.
        ], width={"size":4, "offset":1})
    ]), 

    dbc.Row([
        dbc.Col([
            dcc.RadioItems(['Overview of Germanys medals and their top sports ', 'Germanys accomplishments in the olympics', 'Our explorations about successful parameters for a medal winner..', 'In this country...'], style={'font-size': 20, 'padding-down': 7 }),

        ], width={"size":6, "offset":1} ),

    ]),

    dbc.Row([
        dbc.Col([
            dcc.Checklist(
        ['Sex',   'Countries'], className="mt-2", style={'font-size': 20, "padding-left": 10},
        inline=True)

        ], width={"size":3, "offset":8}),

    ]),
       
    dbc.Row([
        
        dbc.Col([
            dcc.Dropdown(
                 # "id" för att ha en referens när man kopplar funktion/callbacks (?)
                 id="dropdown1",
                 className="text-center mt-1",
                 #False gör att man endast kan välja ett val åt gången. True flera val.  
                 multi=False,
                 options=[data for data in os_data["Sport"].unique()], # Uppdaterad struktur för options
                 # Hexadecimal RGB står färgkoden för. Ändra färg? Googla? Snyggare lista typ lite dimmad eller något? Länkar för välja färg + konvertera till hex. https://www.rapidtables.com/web/color/RGB_Color.html , https://www.rgbtohex.net/
                 style={"color":"#144394"}    
            ),

            # Skapar ett diagram/graf. Id används för att koppla en funktion till den sedan. Om jag använder detta diagrammet i en callback kan jag bara göra tomma måsvingar och ta bort diagram koden här för callbacksen körs alltid först ändå.
            dcc.Graph(
                id="Test_diagram1",
                className="text-center mt-1",
                figure=px.histogram(
                    os_data,
                    x="NOC",
                    y="Age",
                    title="Test Graph with NOC and Ages (original dataset)", 
                    template='plotly_white',
                    labels={"NOC": "NOC", "Age": "Ages"},
                    color="NOC"
                )
            ),

                
            # Bredden på grafen. Halva sidan är width= 6 exempelvis.
        ], width=12),  # Ta bort stängande parentes här

        
    ], justify="evenly")

], fluid=True)






# #Används för att koppla samman något i layouten med funktionen.
# @callback(
#     #I outputens första attribut (ex: Test_diagram) skall id:t stå som föremålet i layouten har. Samt andra attributet: namnet på grafen.
#     Output("Test_diagram", "figure"),
#     #Input är dropdown listan som har id:t som är döpt till "dropdown1", samt "value" kan man sätta till ett värde om man vill att listan ska vara startinställd på något. Hur lägger man till default? Finns i en video. 
#     Input("dropdown1", "value")
# )

# #Skapar en funktion till det som är skapat i layouten. Alltså funktionen retunerar ditt villkor. Callback säger vad som påverkar vad. Dvs när du ändrar i dropdownen "Input", påverkat diagrammet "output".

# def graf_function1():
#     #Retunerar ett nytt diagram, samma kod för diagrammet som står ovan. Nu retunerar den bara det diagrammet med egenskaperna i funktionen. Skriv egna villkor.
#     return px.histogram(os_data,x="NOC", y="Age")



# Debug=False när appen är klar.
app.run(debug=True, jupyter_mode='external', port=230)


Dash app running on http://127.0.0.1:230/
